### Theoretical fundamentals


Let consider the modification to original Kantorovich problem:

$$L_C^\epsilon(a, b) = \min_{\mathbf{P}\in \mathbf{U(a, b)}} \langle \mathbf{P}, \mathbf{C} \rangle - \epsilon\mathbf{H(P)}$$

Here we subtracted entropy of a coupling matrix:

$$\mathbf{H(P)} = -\sum P_{ij}(logP_{ij} - 1)$$ 

Which is strongly concave (**TODO**: find proof), thus as we deduct it from $\langle \mathbf{P}, \mathbf{C} \rangle$ then optimization problem becomes a convex optimization problem.

**Why?**: I belive that $\langle \mathbf{P}, \mathbf{C} \rangle$ is not strictly convex as it a linear function thus spans a hyperplane, and adding a 1-strongly convex entropy necessarily results in strictly convex one.  

As the resulting function is strictly convex it's guarantees existence of one optimal solution, that can be found in the following way using Lagrange multiplier method.

$$\mathcal{L(\mathbf{P}, \mathbf{\lambda}, \mathbf{\gamma})} = \langle \mathbf{P}, \mathbf{C} \rangle - \langle \mathbf{P}\mathbb{1}_m - a, \mathbb{\lambda} \rangle - \langle \mathbf{P}^\top\mathbb{1}_n - b, \mathbb{\gamma} \rangle $$

Searching for stationary points yield the solution if following form:

$$P_{ij}=\exp(\mathbf{\lambda}_i/\epsilon)\exp(- C_{ij}/\epsilon)\exp(\mathbf{\gamma}_j/ \epsilon)$$

If we take all values of first exponent to vector $\mathbf{u}$, last to $\mathbf{v}$, and middle to matrix $\mathbf{K}$, then

$$\mathbf{P}^{*} = diag\{\mathbf{u}\}\mathbf{K}diag\{\mathbf{v}\}$$

Note that this solution must satisfy $P^*\mathbb{1}_m=\mathbf{a}$ and $(\mathbf{P}^*)^\top 1_n=\mathbf{b}$. Those constraints can be rewritten as $\mathbf{u}\odot (\mathbf{Kv}) = \mathbf{a}$ and $\mathbf{v}\odot (\mathbf{K^\top u}) = \mathbf{a}$ (as matrix-vector multiplication is linear combination of matrix column and then the sum is multiplied by $\mathbf{u}_j$ to get the required constraint). Thus, iterative algorithm can be applied. Take $\mathbf{v}^{(0)} = \mathbf{1}$ and update rules

$$\mathbf{u}^{(l+1)} = \frac{\mathbf{a}}{\mathbf{Kv}^{(l)}}$$

$$\mathbf{v}^{(l+1)} = \frac{\mathbf{b}}{\mathbf{K^\top v}^{(l+1)}}$$

It is shown (**TODO:** add proof) that using these iteratioins solution converges.

### Implementation

### Application